# Module 1: Introduction to Exploratory Analysis 

What we'll be doing in this notebook:
-----

 1.  Checking variable type
 2.  Checking for missing variables 
 3.  Number of observations in the dataset
 4.  Descriptive statistics

### Import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import dateutil.parser

# The command below means that the output of multiple commands in a cell will be output at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The command below tells jupyter to display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

### Import dataset

We read in our merged dataset below. Don't forget to update the name with your own! 

In [ ]:
data_path = '../data/'
df = pd.read_csv(data_path+'raw_data.csv.zip', low_memory=False)

In the cell below, we take a random sample of 2 rows to get a feel for the data.

In [ ]:
df.sample(2)

### 1) Type Checking
<a id='type_check'></a>

Type is very important in Python programing, because it affects the types of functions you can apply to a series. There are a few different types of data you will see regularly (see [this](https://en.wikibooks.org/wiki/Python_Programming/Data_Types) link for more detail):
* **int** - a number with no decimal places. example: loan_amount field
* **float** - a number with decimal places. example: partner_id field
* **str** - str is short for string. This type formally defined as a sequence of unicode characters. More simply, string means that the data is treated as a word, not a number. example: sector
* **boolean** - can only be True or False. There is not currently an example in the data, but we will be creating a gender field shortly.
* **datetime** - values meant to hold time data. Example: posted_date

Let's check the type of our variables using the examples we saw in the cell above.

In [ ]:
## Select variables by name
type_example = df[['loan_amount','partner_id', 'sector','posted_date']]
## Pull (3) random rows
type_example.sample(3)

In [ ]:
## Check the first cell for a column
df['posted_datetime'].head(1)
## Check the datatype for a single column
df['posted_datetime'].dtype

Datatype 'O' is for object.

### 2) Do I have missing values?

<a id='missing_check'></a>

If we have missing data, is the missing data at random or not? If data is missing at random, the data distribution is still representative of the population. You can probably ignore the missing values as an inconvenience. However, if the data is systematically missing, the analysis you do may be biased. You should carefully consider the best way to clean the data, it may involve dropping some data.

We want to see how many values are missing in certain variable columns. One way to do this is to count the number of null observations. 

For this, we wrote a short function to apply to the dataframe. 

We print out the first few observations, but you can remove the .head() to print out all columns. 

In [ ]:
#Create a new function:
def num_missing(x):
  return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
## Check how many are missing by column, and then check which ones have any missing values
print(df.apply(num_missing, axis=0).where(lambda x : x != 0).dropna().head(20)) 
#axis=0 defines that function is to be applied on each column

In [ ]:
#Applying per row:
print("\nMissing values per row:")
missing_by_row = df.apply(num_missing, axis=1) 
#axis=1 defines that function is to be applied on each row
print(missing_by_row.head()) 

Remember when we used a left join to merge in the partner data. That means we could have null (missing) values in our partner_id field. We check this in the cell below and find that we have 9,642 loans that are missing. Let's investigate and try and understand whether the data is missing at random or systematically missing.

In [ ]:
null_basket_amount = df.loc[df['basket_amount'].isnull()]
len(null_basket_amount)
null_basket_amount.head(2)

It seems that the number of loans peaked in 2014, and has decreased. All of the loans issued in the second half of 2015, 2016, 2017 have a partner. This tells us these values are not missing at random, they are more likely to occur in data before June 2015, and are most likely to occur in the first half of 2014. We should probably just drop data with the partner_id missing. We do so below using the notnull() function which checks if the field is populated.

In [ ]:
df = df.loc[df['partner_id'].notnull()]

As a sanity check that our filtering was correctly done we check the new number of rows. It makes sense!

In [ ]:
len(df.index)

### 3) Sanity Checks
<a id='obs_check'></a>

**Does the dataset match what you expected to find?**
- is the range of values what you would expect. For example, are all loan_amounts above 0.
- do you have the number of rows you would expect
- is your data for the date range you would expect. For example, is there a strange year in the data like 1880.
- are there unexpected spikes when you plot the data over time


In the command below we find out the number of loans and number of columns by using the function shape. You can also use len(df.index) to find the number of rows.

In [ ]:
print('There are %d observations and %d features' % (df.shape[0],df.shape[1]))

This tells us there are 118,316 observations and 84 features. Remember, each row is an observation and each column is a potential feature. 118,000 matches with what we expect based upon our conversations with Kiva. This is a good sized sample for machine learning.

### 4) Descriptive statistics of the dataset

<a id='desc_stats'></a>

The "describe" command below provides key summary statistics for each numeric column.

In [ ]:
df.describe()

In order to get the same summary statistics for categorical columns (string) we need to do a little data wrangling. The first line of code filters for all columns that are a data type object. As we know from before this means they are considered to be a string. The final row of code provides summary statistics for these character fields.

In [ ]:
categorical = df.dtypes[df.dtypes == "object"].index
df[categorical].describe()

In the table above, there are 4 really useful fields: 

1) **count** - total number of fields populated (Not empty). 

2) **unique** - tells us how many different unique ways this field is populated. For example 4 in description.languages tells us there are 4 different language descriptions. 

3) **top** - tells us the most popular data point. For example, the top activity in this dataset is Farming which tells us most loans are in Farming.

4) **freq** - tells us that how frequent the most popular category is in our dataset. For example, 'en' (English) is the language almost all descriptions (description.languages) are written in (118,306 out of 118,316).

### Moving on

In the next section, we move on to exploratory data analysis. We'll plot this data!

<br>
<br> 
<br>

----